In [14]:
import pandas as pd
import json
from datetime import datetime
df = pd.read_csv("./resources/business_sample.csv")

In [15]:
def parsehours(h):
    if type(h) is float:
        return 0
    total = 0
    all_days = h.split(',')
    for day in all_days:
        times = day.split("-")
        times[0] = times[0].split('\'')[3]
        times[1] = times[1].replace('\'', '')
        times[1] = times[1].replace('}', '')
        time1 = datetime.strptime(times[0], "%H:%M")
        time2 = datetime.strptime(times[1], "%H:%M")
        total += (time2 - time1).seconds/3600
    return total   

In [16]:
clean = df.drop(['Unnamed: 0', 'business_id', 'name', 'address', 'city', 'latitude', 'longitude', 'postal_code', 'hours'], axis=1)
clean['total_hours'] = df['hours'].apply(lambda x: parsehours(x))

In [30]:
clean.to_csv('./resources/business_clean.csv', index=False)

Notes: Dropping 'attributes' cause it looks painful to work with

In [25]:
test = clean.drop(['attributes'], axis=1)
unique_categories = set()

In [26]:
def parse_categories(c):
    categories = c.split(',')
    for category in categories:
        unique_categories.add(category.strip())

In [27]:
test['categories'].apply(lambda x: parse_categories(x))

0      None
1      None
2      None
3      None
4      None
       ... 
195    None
196    None
197    None
198    None
199    None
Name: categories, Length: 200, dtype: object

In [28]:
unique_categories

{'American (New)',
 'American (Traditional)',
 'Argentine',
 'Arts & Entertainment',
 'Asian Fusion',
 'Automotive',
 'Bagels',
 'Bakeries',
 'Banks & Credit Unions',
 'Barbeque',
 'Bars',
 'Beauty & Spas',
 'Bed & Breakfast',
 'Beer',
 'Beer Bar',
 'Breakfast & Brunch',
 'Breweries',
 'Brewpubs',
 'British',
 'Bubble Tea',
 'Buffets',
 'Burgers',
 'Cafes',
 'Cajun/Creole',
 'Caribbean',
 'Caterers',
 'Cheese Shops',
 'Cheesesteaks',
 'Chicken Shop',
 'Chicken Wings',
 'Chinese',
 'Cocktail Bars',
 'Coffee & Tea',
 'Comfort Food',
 'Convenience Stores',
 'Cuban',
 'Delis',
 'Desserts',
 'Diners',
 'Dinner Theater',
 'Do-It-Yourself Food',
 'Empanadas',
 'Ethnic Food',
 'Event Planning & Services',
 'Farmers Market',
 'Fast Food',
 'Financial Services',
 'Florists',
 'Flowers & Gifts',
 'Food',
 'Food Delivery Services',
 'Food Stands',
 'Food Trucks',
 'French',
 'Gas Stations',
 'Gelato',
 'Gluten-Free',
 'Greek',
 'Grocery',
 'Haitian',
 'Halal',
 'Hookah Bars',
 'Hot Dogs',
 'Hotels

In [29]:
len(unique_categories)

126